论文原文：[《Playing Atari with Deep Reinforcement Learning》](https://arxiv.org/abs/1312.5602)

参考学习地址：
* [深度强化学习之深度Q网络DQN详解](https://zhuanlan.zhihu.com/p/145102068)
* [强化学习——从Q-Learning到DQN到底发生了什么？](https://zhuanlan.zhihu.com/p/35882937)

代码库：

Deep Q Network属于DRL（深度强化学习）的一种。Q表当$(s,a)$组合不可穷尽时，无法通过查表选择最优action。DQN实际上总体思路还是使用的Q-learning思路，不过不过对于给定状态选取哪个动作所能得到的Q值，却是由一个深度神经网络来计算的了。

<img style="display: block; margin: 0 auto;" src="../../../assets/images/dqn.png" width = "600" height = "300" align=center />

# 训练流程
DNN的输出值为给定状态$s$下，执行各action可得到的Q值。然后很多时候并不知道最优的Q值是多少，但reward是不变的。因此让预测Q值和真实Q值的比较问题转换成让模型实质上在拟合reward的问题。损失函数：
$$
\begin{align}
L_i(\theta_i) = \mathbb{E}_{s,a,s',r\sim D}(\underbrace{r+\gamma \underset{a'}{max}Q(s',a';\theta_i^-)}_{target} - Q(s,a;\theta_i))^2
\end{align}
$$

其中target部分为这一步得到的reward+下一状态能得到的最大Q值，然后减去这一步的Q值，这个差值也即实际reward减去现有模型认为在$s$下采取$a$时能得到的reward值。这时训练集需要提高一批四元组$(s,a,r,s')$，其中$s'$为执行$a$后的下一状态。由于每次执行一个动作后都能转移到下一个状态，并获得一个reward，因此我们每执行一次动作后都可以获得一个这样的四元组，也可以将这个四元组直接放入经验池中。

这种四元组之间是存在关联性的，因为状态的转移是连续的，如果直接按顺序取一批四元组作为训练集，那么是容易过拟合的，因为训练样本间不是独立的！为解决这个问题，我们可以简单地从经验池中随机抽取少量四元组作为一个batch，这样既保证了训练样本是独立同分布的，也使得每个batch样本量不大，能加快训练速度

<img style="display: block; margin: 0 auto;" src="../../../assets/images/dqn-1.png" width = "600" height = "300" align=center />

DQN需要设计两个DNN，结构一样，参数不同，一个是$\theta^-$，一个是$\theta$。每次迭代更新$\theta$，且规定每运行$C$步后另$\theta^-=\theta$。主要目的是发昂值过拟合，因为若只有一个DNN，不停更新后的目标（$Q(s,a), maxQ(s',a')$）也改变，通过暂时固定target部分，使得更新$\theta$追逐的是一个固定的目标。同时伪代码变为了：

<img style="display: block; margin: 0 auto;" src="../../../assets/images/dqn-2.png" width = "600" height = "400" align=center />
